In [4]:
!pip install galois
!pip install scipy

/bin/bash: line 1: /home/michele/Desktop/Tirocinio_giusto/Tirocinio/.venv/bin/pip: cannot execute: required file not found
/bin/bash: line 1: /home/michele/Desktop/Tirocinio_giusto/Tirocinio/.venv/bin/pip: cannot execute: required file not found


In [1]:
import galois
import numpy as np
import time
import os
from scipy import stats
import csv

In [2]:
#definisco un campo di Galois come nell'esempio 
field = galois.GF(127)

In [3]:
#definisco una matrice k x (n-k)
k = 126 #le righe
n_k = 126 #le colonne

#N.B: Less usa q = dimensione campo: 127; k = 126; n = 252 visto da pdf myLESS_CF pagina 38 di 45

In [19]:
max_column = 12

In [5]:
#number of iteration for Montecarlo simulatiom
num_iterations = 1000

In [23]:
def has_duplicate_rows(matrix):
    _, counts = np.unique(matrix, axis=0, return_counts=True)
    return np.any(counts > 1)

def make_bin_matrix(dim, field):
    bin_matrix = np.zeros(dim, dtype=int)
    perm = np.random.permutation(dim[0])
    for i in range(dim[0]):
        bin_matrix[i, perm[i]] = 1
    return field(bin_matrix)

# order by count number of 0, 1, ..., q in a vector (dictionary). A multiset1 < multiset2 if number of 0 > number of 0 of 
# second multiset. If the number of 0 is the same check number of 1 and so on
def order_row(matrix, error_counter):
    operations = 0
    vector_of_counts = []    
    start = time.perf_counter()
    matrix_int = matrix.view(np.ndarray)
    
    for row in matrix_int:
        counts = [0] * (field.order+1)  
        for value in row:
            #print(value)
            counts[value] += 1
            operations += 1
        vector_of_counts.append(counts)

    vector_of_counts = np.array(vector_of_counts)
    if has_duplicate_rows(vector_of_counts):
        error_counter += 1
        print("Duplicated rows",error_counter)
        return [], 0, 0, [], 0, 0, error_counter
    
    sorted_index = np.lexsort([vector_of_counts[:, i] for i in range(field.order, -1, -1)])[::-1]
    end = time.perf_counter()
    duration = (end - start) * 1000    
     
    operations1 = 0
    vector_of_counts1 = []
    start = time.perf_counter()
    matrix_int = matrix.view(np.ndarray)  
    #print(matrix_int)
    for row in matrix_int:
        #make a small array
        counts1 = [0] * max_column
        for value in row:
            #count only value < max_column
            if value < max_column:
                counts1[value] += 1
                operations1 += 1
        #print(counts1)
        vector_of_counts1.append(counts1)
        #print(vector_of_counts1)
    #print(vector_of_counts1)
    vector_of_counts1 = np.array(vector_of_counts1)
    if has_duplicate_rows(vector_of_counts1):
        error_counter += 1
        print("Duplicated rows",error_counter)
        return [], 0, 0, [], 0, 0, error_counter
    sorted_index1 = np.lexsort([vector_of_counts1[:, i] for i in range(vector_of_counts1.shape[1]-1, -1, -1)])[::-1]
    
    """for raw in matrix[sorted_index1]:
        print(np.sort(raw))"""
    #print()
    end = time.perf_counter()
    duration1 = (end - start) * 1000 
    print(np.array_equal(matrix[sorted_index], matrix[sorted_index1]))
    if np.array_equal(matrix[sorted_index], matrix[sorted_index1]):
        print("If positivo")
        return matrix[sorted_index], duration, operations, matrix[sorted_index1], duration1, operations1, error_counter
    else:
        error_counter += 1
        print(error_counter)
        return [], duration, operations, [], duration1, operations1, error_counter

# order column as always
def sort_by_column(matrix, rows, num_swap_columns):
    #print("Funzione ordina per colonna")
    #tengo posizioni originali per confrontarle con quelle cambiate e quindi vedere quanti swap di riga ho fatto
    initial_position = np.arange(matrix.shape[1])
    
    # Trova gli indici di ordinamento usando np.lexsort sulle righe, se c'è conflitto passa a el seconda riga
    sorting_indices = np.lexsort([matrix[row, :] for row in rows[::-1]])
        
    # Conta gli swap solo quando la posizione delle colonne cambia
    for i, index in enumerate(sorting_indices):
        if index != initial_position[i]:
            num_swap_columns += 1
    
    #riordina la matrice in base agli indici trovati
    sorted_matrix = matrix[:, sorting_indices]
    """for raw in sorted_matrix:
        print(raw)
    print()"""
    return sorted_matrix, num_swap_columns

def algorithm_test(matrix1, matrix2):
    row1_sort, duration1 = order_row(matrix1, 0)
    #print(row1_sort)
    matrix1_sorted = sort_by_column(row1_sort, range(k), 0)
    #print(matrix1_sorted)

    row2_sort, duration2 = order_row(matrix2, 0)
    #print(row2_sort)
    matrix2_sorted = sort_by_column(row2_sort, range(k), 0)
    #print(matrix2_sorted)

    return np.array_equal(matrix1_sorted, matrix2_sorted), duration1, duration2

def algorithm_test1(matrix1, error_counter):
    row1_sort, duration1_1, operations, row1_2_sort, durations1_2, operations1, error_counter = order_row(matrix1, error_counter)
    #print(row1_sort)
    if len(row1_sort) == 0:
        return False, duration1_1, operations, row1_2_sort, durations1_2, operations1, error_counter
    matrix1_sorted = sort_by_column(row1_sort, range(k), 0)
    #print(matrix1_sorted)
    return True, duration1_1, operations, row1_2_sort, durations1_2, operations1, error_counter

def montecarlo_simulation(num_iterations):
    for i in range(num_iterations):
        A = field(np.random.randint(0, field.order, size=(k, n_k))) 
        perm1 = make_bin_matrix((k, n_k), field)
        perm2 = make_bin_matrix((k, n_k), field)
        A_primo = perm1 @ A @ perm2
        check, duration1, duration2 = algorithm_test(A, A_primo)
        if check:
            print(i)



def motecarlo_simulation1(num_iterations):
    vector_operations = []
    vector_operations1 = []
    vector_duration1 = []
    vector_duration1_2 = []
    error_counter = 0
    duration1_accumulator = 0
    duration1_2_accumulator = 0
    operations_accumulator = 0
    operations1_accumulator = 0
    csv_file_path = '/home/michele/Desktop/Tirocinio_giusto/Tirocinio/post_seconda_revisione/dati.csv'
    file_exists = os.path.exists(csv_file_path)
    with open('dati.csv', mode='a', newline='') as file:
        writer_csv = csv.writer(file)
        if not file_exists:
            writer_csv.writerow(['File','Iteration', 'Max_columns', 'Duration_w_all_Columns', 'Duration_w_max_columns', 'Num_op_w_all_columns', 'Num_op_w_max_columns'])

        for i in range(num_iterations):
            A = field(np.random.randint(0, field.order, size=(k, n_k))) 
            while(has_duplicate_rows(A)):
                A = field(np.random.randint(0, field.order, size=(k, n_k))) 


            check, duration1_1, operations, row1_2_sort, durations1_2, operations1, error_counter = algorithm_test1(A, error_counter)
            if check:
                duration1_accumulator += duration1_1
                duration1_2_accumulator += durations1_2
                operations_accumulator += operations
                operations1_accumulator += operations1
                vector_operations.append(operations)
                vector_operations1.append(operations1)
                vector_duration1.append(duration1_1)
                vector_duration1_2.append(durations1_2)
                writer_csv.writerow([4,i, max_column, duration1_1, durations1_2, operations, operations1])

    t_stat, p_value = stats.ttest_rel(vector_operations, vector_operations1)
    print(f"T-statistica: {t_stat}, P-value: {p_value}")
    alpha = 0.05
    if p_value < alpha:
        print("La differenza tra operazioni normali e sufficienti è statisticamente significativa.")
    else:
        print("La differenza non è significativa.")

    t_stat1, p_value1 = stats.ttest_rel(vector_duration1, vector_duration1_2)
    print(f"T-statistica: {t_stat1}, P-value: {p_value1}")
    if p_value1 < alpha:
        print("La differenza tra durate normali e sufficienti è statisticamente significativa.")
    else:
        print("La differenza non è significativa.")
            
    return duration1_accumulator, duration1_2_accumulator, operations_accumulator, operations1_accumulator, p_value, (num_iterations-error_counter)/num_iterations, p_value1


In [64]:
#ora fai un po di statistiche e prova a limitare numero di colonne come nel caso calassico
montecarlo_simulation(num_iterations)

TypeError: list indices must be integers or slices, not tuple

In [24]:
duration1_accumulator, duration1_2_accumulator, operations_accumulator, operations1_accumulator, p_value, good_perc, p_value1 = motecarlo_simulation1(num_iterations)

True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positivo
True
If positi

In [25]:
print("Duration1_accumulator: ", duration1_accumulator)
print("Duration1_2_accumulator: ", duration1_2_accumulator)
print("Operations_accumulator: ", operations_accumulator)
print("Operations1_accumulator: ", operations1_accumulator)
print("P-value: ", p_value)
print("Good_perc: ", good_perc)
print("P_value1: ", p_value1)

Duration1_accumulator:  20626.0353659909
Duration1_2_accumulator:  10037.563600003978
Operations_accumulator:  15796620
Operations1_accumulator:  1492167
P-value:  0.0
Good_perc:  0.995
P_value1:  6.1569528977171315e-27


In [26]:
#print some stats
nome_file = "stat_without_sort.txt"

with open(nome_file, 'a') as file:
    file.write(f"Qui c'è il limite colonne a: {max_column} file 3\n")
    file.write(f"Tempo esecuzione con tutte colonne: {duration1_accumulator} ms\n")
    file.write(f"Tempo esecuzione con {max_column} colonne: {duration1_2_accumulator} ms\n")
    file.write(f"Numero operazioni con tutte colonne: {operations_accumulator}\n")
    file.write(f"Numero operazioni con {max_column} colonne: {operations1_accumulator}\n")
    file.write(f"P-value per operazioni: {p_value}\n")
    file.write(f"P-value per tempi: {p_value1}\n")
    file.write(f"Percentuale test positivi (tutte colonne = meno colonne): {good_perc}%\n\n")